In [ ]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize

import string 

from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag, ne_chunk
from nltk.probability import FreqDist
from nltk.cluster.util import cosine_distance

from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
import os
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import streamlit as st

: 

In [ ]:
stemmer = SnowballStemmer("english")
punct = string.punctuation

In [1]:
# text from input user
# text = str(input())
text = "Apple Inc. is an American multinational technology company headquartered in Cupertino, California that designs, develops, and sells consumer electronics, computer software, and online services. The company's hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player, the Apple smartwatch, and the Apple TV digital media player."

## Preprocess(token, remove stopword, stemming)

In [4]:
stopword= set(stopwords.words('english'))
sentences = sent_tokenize(text)

def preprocess_text(sentence):
    # tokenization
    words = word_tokenize(sentence)
    # punctuation character
    rm_punct = [word for word in sentence if word not in punct]
    # rm stopword
    rm_stopword= [ w for w in words if w.lower() not in stopword]

    # stemming
    stemmed = [stemmer.stem(w) for w in rm_stopword]

    return stemmed

In [5]:
print(sentences)

['Apple Inc. is an American multinational technology company headquartered in Cupertino, California that designs, develops, and sells consumer electronics, computer software, and online services.', "The company's hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player, the Apple smartwatch, and the Apple TV digital media player."]


In [7]:

# preprocess sentence
preprocessed_sentences =[preprocess_text(sentence) for sentence in sentences]
print("preprocessed : ", preprocessed_sentences)
# flatten list preprocess
flat_preprocessed_word = [word for sentence in preprocessed_sentences for word in sentence]

print(flat_preprocessed_word)
# word freq
word_freq = FreqDist(flat_preprocessed_word)

print(word_freq)

preprocessed :  [['appl', 'inc.', 'american', 'multin', 'technolog', 'compani', 'headquart', 'cupertino', ',', 'california', 'design', ',', 'develop', ',', 'sell', 'consum', 'electron', ',', 'comput', 'softwar', ',', 'onlin', 'servic', '.'], ['compani', "'s", 'hardwar', 'product', 'includ', 'iphon', 'smartphon', ',', 'ipad', 'tablet', 'comput', ',', 'mac', 'person', 'comput', ',', 'ipod', 'portabl', 'media', 'player', ',', 'appl', 'smartwatch', ',', 'appl', 'tv', 'digit', 'media', 'player', '.']]
['appl', 'inc.', 'american', 'multin', 'technolog', 'compani', 'headquart', 'cupertino', ',', 'california', 'design', ',', 'develop', ',', 'sell', 'consum', 'electron', ',', 'comput', 'softwar', ',', 'onlin', 'servic', '.', 'compani', "'s", 'hardwar', 'product', 'includ', 'iphon', 'smartphon', ',', 'ipad', 'tablet', 'comput', ',', 'mac', 'person', 'comput', ',', 'ipod', 'portabl', 'media', 'player', ',', 'appl', 'smartwatch', ',', 'appl', 'tv', 'digit', 'media', 'player', '.']
<FreqDist with 3

## Extractive

In [ ]:
# abstractive -> adding sentence other than the input text
# extractive -> summarize text dari input

# score -> how many word appear in sentence
def scores_sentences(sentences, word_freq):
    sentence_scores={}

    for i, sentence in enumerate(sentences):
        for word in sentence:
            if word in word_freq:
                if i in sentence_scores:
                    sentence_scores[i] += word_freq[word]
                else:
                    sentence_scores[i] = word_freq[word]
    return sentence_scores

sentence_scores = scores_sentences(preprocessed_sentences, word_freq)
print(sentence_scores)

{0: 75, 1: 89}


In [14]:
# generate summary by select top sentence

summary_sentences=[]
if sentence_scores:
    sorted_scores = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)
    top_sentences = sorted_scores[:3]
    print(top_sentences)
    for index, _ in top_sentences:
        summary_sentences.append(sentences[index])

print(summary_sentences)
# join summary
summary = ''.join(summary_sentences)

print('SUMMARY')
print(summary)

[(1, 89), (0, 75)]
["The company's hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player, the Apple smartwatch, and the Apple TV digital media player.", 'Apple Inc. is an American multinational technology company headquartered in Cupertino, California that designs, develops, and sells consumer electronics, computer software, and online services.']
SUMMARY
The company's hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player, the Apple smartwatch, and the Apple TV digital media player.Apple Inc. is an American multinational technology company headquartered in Cupertino, California that designs, develops, and sells consumer electronics, computer software, and online services.


## abstractive

In [15]:
from transformers import pipeline 

In [ ]:
summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

summary_abs = summarizer(text, max_length=50,
                         min_length=25,
                         do_sample=False) # greedy search(token highest probability) - output stabil(sama)
result_abs=summary_abs[0]['summary_text']
print(result_abs)

Device set to use cpu


Apple Inc. designs, develops, and sells consumer electronics, computer software, and online services. Its hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, and the iPod portable media player.


In [ ]:
# %pip install rouge_score absl-py

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=25027 sha256=1223eea1451be04417ed2316c2540bedcaaf046724291dced91a6e9a1baf28b9
  Stored in directory: c:\users\graze\appdata\local\pip\cache\wheels\1e\19\43\8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import evaluate
rouge = evaluate.load('rouge')
result = rouge.compute(predictions=[result_abs], references=[summary]) # kemiripan dengan extractive 42%
print(result)

{'rouge1': 0.7333333333333333, 'rouge2': 0.6590909090909091, 'rougeL': 0.4222222222222223, 'rougeLsum': 0.4222222222222223}
